In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import split, trim, col, replace

In [0]:

spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 10 * 1024 * 1024)

In [0]:
# paths
path_silver = "abfss://silver@storagedatanexus.dfs.core.windows.net/"
dim_isic_table = "gold.dim_ncm_isic"
dim_pais_table = "gold.dim_pais"
dim_calendario_table = "gold.dim_calendario"
dim_cnaes_table = "gold.dim_cnaes"
dim_uf_table = "gold.dim_uf"

##Read das tabelas na silver

In [0]:
df_dim_pais = spark.read.format("delta").load(path_silver + "antigo_landingbeca2026jan/" + "comercio_ext_auxiliares/" + "tb_auxiliar_paises") \
    .select(
        F.col("CO_PAIS").alias("cod_pais"), 
        F.col("NO_PAIS").alias("pais"),
    )

df_dim_bloco_economico = spark.read.format("delta").load(path_silver + "antigo_landingbeca2026jan/" + "comercio_ext_auxiliares/" + "tb_referencia_pais_bloco") \
    .select(
        F.col("CO_PAIS").alias("cod_pais"), 
        F.col("NO_BLOCO").alias("bloco_economico"),
    )

df_dim_isic_ncm = spark.read.format("delta").load(path_silver + "antigo_landingbeca2026jan/" + "comercio_ext_indices/" + "tb_nomenclatura_mercosul") \
    .select(
        "CO_NCM", 
        "CO_ISIC_CLASSE",
        "NO_NCM_POR"
    )

df_dim_isic = spark.read.format("delta").load(path_silver + "antigo_landingbeca2026jan/" + "comercio_ext_indices/" + "tb_referencia_ncm_isic") \
    .select(
        "CO_ISIC_CLASSE",
        "CO_ISIC_DIVISAO",
        "NO_ISIC_DIVISAO",
        "NO_ISIC_SECAO"
    )

df_dim_cnaes = spark.read.format("delta").load(path_silver + "antigo_cnaes") \
    .select(
        F.col("cod_cnae"), 
        F.col("descricao"),
    )
df_dim_cnaes.dropDuplicates(["cod_cnae"])

df_dim_uf = (
    spark.read.format("delta")
    .load(path_silver + "antigo_landingbeca2026jan/comercio_ext_auxiliares/tb_auxiliar_unidades_federativas")
    .select(
        F.col("CO_UF").alias("cod_uf"),
        F.col("SG_UF").alias("sigla_uf"),
        F.col("NO_UF").alias("nome_uf"),
        F.col("NO_REGIAO").alias("regiao")
    )
)

df_dim_uf = df_dim_uf.withColumn(
    "regiao",
    F.trim(F.regexp_replace(F.col("regiao"), "REGIAO", ""))
)

##Criação das tabelas dim

In [0]:
# 2) Tratamento do nome do produto: pega texto antes da primeira vírgula
df_dim_isic_ncm_tratado = (
    df_dim_isic_ncm
    .withColumn("NO_NCM_POR", trim(split(col("NO_NCM_POR"), ",").getItem(0)))
)

# 3) Dedup para evitar explosão no join
dim_isic_ncm = df_dim_isic_ncm_tratado.dropDuplicates(["CO_NCM"])
dim_isic_div = df_dim_isic.dropDuplicates(["CO_ISIC_CLASSE"])

# 4) Join + seleção final com aliases
dim_ncm_isic = (
    dim_isic_ncm.alias("ncm")
    .join(dim_isic_div.alias("isic"), on="CO_ISIC_CLASSE", how="left")
    .select(
        F.col("ncm.CO_NCM").alias("cod_ncm"),
        F.col("ncm.CO_ISIC_CLASSE").alias("cod_isic"),
        F.col("isic.CO_ISIC_DIVISAO").alias("cod_div_isic"),
        F.col("ncm.NO_NCM_POR").alias("nome_produto"),
        F.col("isic.NO_ISIC_DIVISAO").alias("div_isic"),
        F.col("isic.NO_ISIC_SECAO").alias("setor_produto"),
    )
)

In [0]:
# 1) Dedup nas fontes
dim_pais = df_dim_pais.dropDuplicates(["cod_pais"])
dim_bloco = df_dim_bloco_economico.dropDuplicates(["cod_pais"])

# 2) Join para enriquecer a dimensão País
dim_pais = (
    dim_pais.alias("p")
    .join(
        dim_bloco.alias("b"),
        on="cod_pais",
        how="left"  
    )
    .select(
        F.col("p.cod_pais"),
        F.col("p.pais"),
        F.col("b.bloco_economico")
    )
)


In [0]:
dim_calendario = spark.sql("""
    SELECT
        ano,
        mes,
        concat(ano, '-', lpad(mes, 2, '0')) AS ano_mes
    FROM (
        SELECT explode(sequence(2020, 2030)) AS ano
    ) a
    CROSS JOIN (
        SELECT explode(sequence(1, 12)) AS mes
    ) m
""")

##Write na GOLD

In [0]:
dim_pais.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwrite", "true") \
    .saveAsTable(dim_pais_table)

In [0]:
dim_calendario.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwrite", "true") \
    .saveAsTable(dim_calendario_table)

In [0]:
dim_ncm_isic.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwrite", "true") \
    .saveAsTable(dim_isic_table)

In [0]:
df_dim_cnaes.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwrite", "true") \
    .saveAsTable(dim_cnaes_table)

In [0]:
df_dim_uf.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwrite", "true") \
    .saveAsTable(dim_uf_table)